In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import numpy as np

In [2]:
def embed_message(image_path, message, output_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    pixels = np.array(image)
    flat_pixels = pixels.flatten()
    
    message += '###'  # Mesaj sonu işareti
    message_bits = ''.join(f'{ord(c):08b}' for c in message)
    if len(message_bits) > len(flat_pixels):
        raise ValueError("Mesaj çok uzun, görüntüye sığmıyor.")
    
    for i, bit in enumerate(message_bits):
        flat_pixels[i] = (flat_pixels[i] & ~1) | int(bit)
    
    new_pixels = flat_pixels.reshape(pixels.shape)
    new_image = Image.fromarray(new_pixels.astype('uint8'), 'RGB')
    new_image.save(output_path)

def extract_message(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    pixels = np.array(image)
    flat_pixels = pixels.flatten()
    
    bits = ''
    for i in range(len(flat_pixels)):
        bits += str(flat_pixels[i] & 1)
        if bits.endswith('001000110010001100100011'):  # '###' bit dizisi
            message = ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits)-24, 8))
            return message
    return ''

In [ ]:
def select_image():
    path = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])
    entry_image.delete(0, tk.END)
    entry_image.insert(0, path)

def save_image():
    path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
    entry_output.delete(0, tk.END)
    entry_output.insert(0, path)

def hide_message():
    image_path = entry_image.get()
    output_path = entry_output.get()
    message = entry_message.get()
    if len(message) > 160:
        messagebox.showerror("Hata", "Mesaj 160 karakterden uzun olamaz.")
        return
    try:
        embed_message(image_path, message, output_path)
        messagebox.showinfo("Başarılı", "Mesaj başarıyla gizlendi.")
    except Exception as e:
        messagebox.showerror("Hata", str(e))

def reveal_message():
    image_path = entry_image.get()
    try:
        message = extract_message(image_path)
        messagebox.showinfo("Gizli Mesaj", message)
    except Exception as e:
        messagebox.showerror("Hata", str(e))

# Arayüz oluştur
root = tk.Tk()
root.title("Sezgisel Steganografi")

tk.Label(root, text="Giriş Görüntüsü:").grid(row=0, column=0, sticky="e")
entry_image = tk.Entry(root, width=50)
entry_image.grid(row=0, column=1)
tk.Button(root, text="Seç", command=select_image).grid(row=0, column=2)

tk.Label(root, text="Çıkış Görüntüsü:").grid(row=1, column=0, sticky="e")
entry_output = tk.Entry(root, width=50)
entry_output.grid(row=1, column=1)
tk.Button(root, text="Kaydet", command=save_image).grid(row=1, column=2)

tk.Label(root, text="Mesaj:").grid(row=2, column=0, sticky="e")
entry_message = tk.Entry(root, width=50)
entry_message.grid(row=2, column=1, columnspan=2)

tk.Button(root, text="Mesajı Gizle", command=hide_message).grid(row=3, column=1)
tk.Button(root, text="Mesajı Göster", command=reveal_message).grid(row=3, column=2)

root.mainloop()